In [1]:
import pandas as pd
import glob
import csv
import os

In [2]:
maquinas = glob.glob('maquinas/*.csv')
nomes = pd.read_csv('nomes/nomes_maquinas.csv')

In [24]:
def get_description(maquina):
    df = pd.read_csv(maquina)
    return df.iloc[0]['Descrição']
    
def check_names(nome_lista, maquina_atual):
    return nome_lista in maquina_atual

def filter_name(nome_lista):
    return nome_lista.replace(";", "")

In [25]:
if not os.path.isfile('./lista.csv'): # Caso o arquivo não exista, cria e adiciona o cabeçalho
    with open('lista.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Nome', 'Descricao'])


In [26]:
with open('lista.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(nomes.shape[0]):
        nome_puro = filter_name(nomes.iloc[i][0])

        for j in range(len(maquinas)):
            if check_names(nome_puro, maquinas[j]) == True:
                descricao = get_description(maquinas[j])
                filewriter.writerow([nome_puro,descricao])
                break
        else:
            descricao = "fora da lista"
            filewriter.writerow([nome_puro,descricao])
